<a href="https://colab.research.google.com/github/Web-Jose/CSCI-167/blob/main/ValueIteration_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Frozen Lake Domain Description

Frozen Lake is a simple grid-world environment where an agent navigates a frozen lake to reach a goal while avoiding falling into holes. The environment is represented as a grid, with each cell being one of the following:

* **S**: Starting position of the agent
* **F**: Frozen surface, safe to walk on
* **H**: Hole, falling into one ends the episode with a reward of 0
* **G**: Goal, reaching it ends the episode with a reward of 1

The agent can take four actions:

* **0: Left**
* **1: Down**
* **2: Right**
* **3: Up**

However, due to the slippery nature of the ice, the agent might not always move in the intended direction. There's a chance it moves perpendicular to the intended direction.




In [4]:
import gym

# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')  # 'ansi' mode for text-based rendering

# Reset the environment to the initial state
observation = env.reset()

# Take a few actions and observe the results
for _ in range(5):
    action = env.action_space.sample()  # Choose a random action
    observation, reward, done, info = env.step(action)
    # Render the environment to see the agent's movement (text-based)
    if done:
        observation= env.reset()
    else:
      rendered = env.render()
      if len(rendered) > 1:  # Check if there's a second element
         print(rendered[1])  # Print the second element
# Close the environment
env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG



The transition model for the Frozen Lake world describes how the agent's actions affect its movement and the resulting state transitions. Here's a breakdown of the key components:

**Actions:**

* The agent can choose from four actions:
    * 0: Left
    * 1: Down
    * 2: Right
    * 3: Up

**State Transitions:**

* **Intended Movement:** Ideally, the agent moves one cell in the chosen direction.
* **Slippery Ice:** Due to the slippery nature of the ice, there's a probability that the agent will move in a perpendicular direction instead of the intended one. The exact probabilities depend on the specific Frozen Lake environment configuration, but typically:
    * **Successful Move:** The agent moves in the intended direction with a high probability.
    * **Perpendicular Move:** The agent moves 90 degrees to the left or right of the intended direction with a lower probability.
* **Boundaries:** If the intended movement would take the agent outside the grid boundaries, it remains in its current position.
* **Holes:** If the agent lands on a hole ("H"), the episode ends, and it receives a reward of 0.
* **Goal:** If the agent reaches the goal ("G"), the episode ends, and it receives a reward of 1.




In [5]:
import gym

# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')  # 'ansi' mode for text-based rendering

# Reset the environment to the initial state
observation = env.reset()

# Take a few actions and observe the results
for _ in range(5):
    action = env.action_space.sample()  # Choose a random action
    observation, reward, done, info = env.step(action)
    # Render the environment to see the agent's movement (text-based)
    if done:
        observation= env.reset()
    else:
      rendered = env.render()
      if len(rendered) > 1:  # Check if there's a second element
         print(rendered[1])  # Print the second element
# Close the environment
env.close()
print ("State 14 Going Right: (s, a, r, Done)", env.P[14][2])

  (Left)
SFFF
FHFH
FFFH
HFFG

  (Down)
SFFF
FHFH
FFFH
HFFG

State 14 Going Right: (s, a, r, Done) [(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]


In [6]:
import gym
import numpy as np

# Create FrozenLake environment
env = gym.make("FrozenLake-v1")

# Starter code for students (modified for number of iterations)
def value_iteration(env, gamma=0.9, num_iterations=1000):
    """
    Implements the Value Iteration algorithm.

    Args:
        env: The OpenAI Gym environment.
        gamma: Discount factor.
        num_iterations: Number of iterations to run.

    Returns:
        The optimal value function and policy.
    """

    # Initialize value function and policy
    V = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n, dtype=int)

    # TODO: Implement the core Value Iteration logic here
    # - Iterate for 'num_iterations'
    # - For each state:
    #   - Calculate Q values for all actions
    #   - Update V[s] with the max Q value
    #   - Update policy[s] with the action that maximizes Q value

    for i in range(num_iterations):
        new_V = np.copy(V)  # Keep track of updates to state values

        for state in range(env.observation_space.n):
            Q_values = []  # Track Q values for all possible actions from the current state

            # Calculate the Q-value for each action
            for action in range(env.action_space.n):
                q_value = 0

                # Look at the possible next states and rewards from the transition probabilities
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * V[next_state])

                Q_values.append(q_value)

            # Update the value of the current state
            new_V[state] = max(Q_values)  # Value is the maximum Q value for the best action

        # Check for convergence (small change in value function)
        if np.max(np.abs(new_V - V)) < 1e-6:
            break

        V = new_V

    # Derive the optimal policy from the optimal value function
    for state in range(env.observation_space.n):
        Q_values = []
        for action in range(env.action_space.n):
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * V[next_state])
            Q_values.append(q_value)

        # Choose the action with the highest Q value as the optimal action for the policy
        policy[state] = np.argmax(Q_values)

    return V, policy

# Apply student's Value Iteration
optimal_V, optimal_policy = value_iteration(env)

# Display the results of the optimal value function and policy
print("Optimal Value Function:")
print(optimal_V)

print("\nOptimal Policy (0: Left, 1: Down, 2: Right, 3: Up):")
print(optimal_policy)

# Evaluate student's solution (Optional)
def evaluate_policy(env, policy, num_episodes=100):
    total_reward = 0
    for _ in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            action = policy[state]
            state, reward, done, _ = env.step(action)
            total_reward += reward
    return total_reward / num_episodes

average_reward = evaluate_policy(env, optimal_policy)
print("Average Reward:", average_reward)

Optimal Value Function:
[0.06888376 0.06140851 0.07440534 0.05580246 0.09184804 0.
 0.11220583 0.         0.14543109 0.24749341 0.2996151  0.
 0.         0.37993334 0.63901881 0.        ]

Optimal Policy (0: Left, 1: Down, 2: Right, 3: Up):
[0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]
Average Reward: 0.79
